import sys
!{sys.executable} -m pip list 



import sys
!{sys.executable} -m pip install tensorflow


import sys
!{sys.executable} -m pip install keras



import sys
!{sys.executable} -m pip install nltk


import nltk
nltk.download('punkt')
nltk.download('wordnet')
nltk.download('universal_tagset')
nltk.download('spanish_grammars')
nltk.download('tagsets')
nltk.download('stopwords')
nltk.download('omw-1.4')


In [1]:
import json

def guardar_json(datos):
    archivo=open("intents.json","w")
    json.dump(datos,archivo,indent=4)

biblioteca={"intents":
            [
                {"tag":"saludos",
                 "patterns":["hola",
                             "buenos dias",
                             "buenos días",
                             "buen dia",
                             "buen día",
                             "buenas tardes",
                             "buenas noches",
                             "como estas",
                             "cómo estás",
                             "hay alguien ahi?",
                             "hay alguien ahí?",
                             "hey",
                             "saludos",
                             "que tal",
                             "qué tal",
                              "buenas",
                              "que xopa",
                              "holi"
                             
                             ],
                 "responses":["hola soy P-BOT , en qué puedo ayudarte?"
                             ],
                 "context":[""]
                 },
                
                {"tag":"despedidas",
                 "patterns":["chao",
                             "adios",
                             "adiós",
                             "hasta luego",
                             "bye",
                             "hasta pronto",
                             "hasta la proxima", 
                             "hasta la próxima",
                              "eso es todo"
                             ],
                 "responses":["Hasta luego!"
                             ],
                 "context":[""]},
                {"tag":"agradecimientos",
                 "patterns":["gracias",
                             "muchas gracias",
                             "mil gracias",
                             "muy amable",
                             "te lo agradezco",
                             "fue de ayuda",
                             "gracias por la ayuda",
                             "muy agradecido",
                             "gracias por su tiempo",
                             "thank you", 
                             "thanks"
                            ],
                 "responses":["Fue un placer ayudarle."
                             ],
                 "context":[""]
                },
                {"tag":"norespuesta",
                 "patterns":[""],
                 "responses":["No se detectó una respuesta"],
                 "context":[""]                    
                }, 
                {"tag":"ayuda",
                 "patterns":["ayudame", "ayúdame", "necesito", "favor", "quiero", "me gustaría", "me gustaria"],
                 "responses":["Estoy aquí para asistirte."],
                 "context":[""]                    
                },
                {"tag":"afirmación",
                 "patterns":["si", "perfecto", "claro", "exacto", "positivo", "está bien"],
                 "responses":["Me alegra que mi respuesta te haya servido."],
                 "context":[""]                    
                },
                {"tag":"negación",
                 "patterns":["no", "claro que no", "negativo"],
                 "responses":["Oh no!"],
                 "context":[""]                    
                },
                {"tag":"acción_add",
                 "patterns":["ordenar", "deseo", "agregar", "anexar", "pedir" ],
                 "responses":["Qué producto te gustaría agregar al carrito?"],
                 "context":[""]                    
                },
                 {"tag":"acción_delete",
                 "patterns":["eliminar", "borrar", "quitar" ],
                 "responses":["Qué producto te gustaría eliminar del carrito?"],
                 "context":[""]                    
                },
                {"tag":"categoría_rest",
                 "patterns":["comida", "restaurante", "pizza", "hamburguesa", "alimentos", "aperitivo", "almuerzo", "cena", "brunch", "desayuno", "café"],
                 "responses":["Qué se te antoja?", "Tenemos opciones deliciosas"],
                 "context":[""]                    
                },
                {"tag":"categoría_farm",
                 "patterns":["medicina", "farmacia", "farmacéutico", "medicamentos", "fármacos", "enfermedad", "siente mal", "malestar"],
                 "responses":["Lo siento mucho. Tenemos opciones que te harán sentir mejor.", "Que te mejores pronto"],
                 "context":[""]                    
                },
                 {"tag":"categoría_compras",
                 "patterns":["supermercado", "super", "rey", "hacer las compras"],
                 "responses":["PedidosYa Market te lo lleva a tu puerta.", "Dinos tu supermercado de preferencia."],
                 "context":[""]                    
                }
                
                
            ] 
        }
guardar_json(biblioteca)

In [2]:
import json
import pickle
import numpy as np

import nltk
from nltk.stem import SnowballStemmer
from tensorflow.keras import Sequential
from tensorflow.keras.layers import Conv2D, Flatten, Dense,Dropout
from tensorflow.keras.optimizers import SGD


   
stemmer = SnowballStemmer('spanish')

ignore_words=["?","¿","!","¡"]
data_file= open("intents.json").read() 
intents = json.loads(data_file)


In [3]:
words=[]
classes=[]
documents=[]

for intent in intents["intents"]:
    for pattern in intent["patterns"]:
        
        
        w=nltk.word_tokenize(pattern) 
        words.extend(w)
        documents.append((w,intent["tag"]))
        
        if intent["tag"] not in classes:
            classes.append(intent["tag"])
            


In [4]:
words = [stemmer.stem(w.lower())  
for w in words  if w not in ignore_words ] 

pickle.dump(words,open("words.pkl","wb"))
pickle.dump(classes,open("classes.pkl","wb"))




In [5]:
training=[]
output_empty=[0]*len(classes)

for doc in documents: 
    bag=[]
    pattern_words=doc[0]
    # lematizacion del token
    pattern_words= [stemmer.stem(word.lower()) for word in pattern_words  if word not in ignore_words ]
    
    for palabra in words:
        bag.append(1) if palabra in pattern_words else bag.append(0) 
    
    output_row =list(output_empty)
    output_row[classes.index(doc[1])] = 1 
    
    training.append([bag,output_row])

training=np.array(training)


C:\Users\19546\AppData\Local\Temp\ipykernel_2148\2909194545.py:18: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  training=np.array(training)


In [6]:
x_train= list(training[:,0]) 
y_train= list(training[:,1]) 


In [7]:
import tensorflow.compat.v2 as tf

model = Sequential()
model.add(Dense(128, input_shape=(len(x_train[0]),), activation='relu')) 
model.add(Dropout(0.5))
model.add(Dense(64,activation='relu')) 
model.add(Dropout(0.5))
model.add(Dense(len(y_train[0]),activation='softmax')) 

sgd= tf.keras.optimizers.legacy.SGD(learning_rate=0.1)

model.compile(loss="categorical_crossentropy",optimizer=sgd,metrics=["accuracy"])
hist=model.fit(np.array(x_train),np.array(y_train),epochs=300,batch_size=5,verbose=1)
model.save("chatbot_model.h5",hist)
print("modelo creado")
    


Epoch 1/300
17/17 [==============================] - 2s 4ms/step - loss: 2.4673 - accuracy: 0.1294
Epoch 2/300
17/17 [==============================] - 0s 4ms/step - loss: 2.2839 - accuracy: 0.2824
Epoch 3/300
17/17 [==============================] - 0s 3ms/step - loss: 2.1351 - accuracy: 0.2824
Epoch 4/300
17/17 [==============================] - 0s 3ms/step - loss: 2.0870 - accuracy: 0.3059
Epoch 5/300
17/17 [==============================] - 0s 3ms/step - loss: 1.9693 - accuracy: 0.3294
Epoch 6/300
17/17 [==============================] - 0s 3ms/step - loss: 1.9318 - accuracy: 0.3059
Epoch 7/300
17/17 [==============================] - 0s 3ms/step - loss: 1.8300 - accuracy: 0.3765
Epoch 8/300
17/17 [==============================] - 0s 3ms/step - loss: 1.7891 - accuracy: 0.4000
Epoch 9/300
17/17 [==============================] - 0s 4ms/step - loss: 1.7013 - accuracy: 0.4353
Epoch 10/300
17/17 [==============================] - 0s 3ms/step - loss: 1.6806 - accuracy: 0.4471
Epoch 11/

In [8]:
import nltk, json,pickle
import numpy as np

from nltk.stem import SnowballStemmer
from tensorflow.keras.models import load_model
stemmer = SnowballStemmer('spanish')


model=load_model("chatbot_model.h5")
intents= json.loads(open("intents.json").read())
words=pickle.load(open("words.pkl","rb"))
classes=pickle.load(open("classes.pkl","rb"))

In [9]:
def clean_up_sentence(sentence):
    sentence_words=nltk.word_tokenize(sentence)
    sentence_words=[stemmer.stem(word.lower()) for word in sentence_words]
    return sentence_words


def bow (sentence,words,show_details=True): 
    sentence_words=clean_up_sentence(sentence)
    
    bag=[0]*len(words)
    
    for i in sentence_words:
        for j,w in enumerate(words):
            if w==i: 
                bag[j]=1
    return (np.array(bag))



def predict_class(sentence,model):
    p = bow(sentence,words,show_details=False) 
    res = model.predict(np.array([p]))[0] 
    ERROR_THRESHOLD=0.25 
    

    results= [[i,r] for i,r in enumerate(res) if r>ERROR_THRESHOLD] 
    
    results.sort(key=lambda x: x[1], reverse=True)  
    return_list = []    
    for r in results:   
        return_list.append({"intent": classes[r[0]], "probability": str(r[1])})   
    return return_list 

In [10]:
import random

def get_response(ints,intents_json): 
    tag= ints[0]["intent"] 
    list_of_intents=intents_json["intents"] 
    
    for i  in list_of_intents: 
        if (i["tag"]==tag): 
            result= random.choice(i["responses"]) 
            break
    return result
    
def chatbot_response(text): 
    ints=predict_class(text,model) 
    res=get_response(ints,intents)
    return res

texto_us="" 
print(" bienvenido, para salir  escriba salir \n")

while texto_us!="salir":
    texto_us=input()
    res=chatbot_response(texto_us)
    print(res)

 bienvenido, para salir  escriba salir 

compra
1/1 [==============================] - 0s 205ms/step
PedidosYa Market te lo lleva a tu puerta.
salir
1/1 [==============================] - 0s 31ms/step
No se detectó una respuesta



#TENGO QUE TERMINAR ESTA PARTE, ESPERARÉ AL FINAL PARA HACER LA INTERFAZ COMPLETA


from tkinter import * 
def send():
    msg=entrybox.get("1.0","end-1c").strip() 
    entrybox.delete("0.0",END)
    if msg!="":
        chatlog.config(state=NORMAL)
        chatlog.insert(END,"USUARIO:" + msg +"\n\n" )
        chatlog.config(foreground="#442265",font=("verdana",12))
        res=chatbot_response(msg)
        chatlog.insert(END,"P-BOT:" + res +"\n\n" )
        chatlog.config(state=DISABLED)
        chatlog.yview(END)

base=Tk() 
base.title("P-BOT")
base.geometry("400x500") 
base.resizable(width=False,height=False)


chatlog= Text(base,bd=0,bg="white",height="8",width="50",font="Arial")
#METER EL OBJETO CHATLOG EN LA VENTANA, CREO QUE LO MISMO HABRIA QUE HACER CON LOS BOTONES DE AGREGAR CARRITO Y ESO
chatlog.config(state=DISABLED)

scrollbar = Scrollbar(base,command=chatlog.yview, cursor="heart")
chatlog["yscrollcommand"]=scrollbar.set

# casilla para introducir el mensaje
entrybox=Text(base,bd=0,bg="white",height="5",width="26",font="Arial")

# boton para enviar el mensaje
sendbutton= Button(base, font=("Verdana",12,'bold'),
                    text="Enviar", width="10", height=5, bd=0, bg="#32de97",
                    activebackground="#3c9d9b",fg='#ffffff', command= send )

# invoco, coloco los componentes en la pantalla
scrollbar.place(x=376,y=6, height=386)
chatlog.place(x=6,y=6, height=386, width=370)
entrybox.place(x=6, y=401, height=90, width=265)
sendbutton.place(x=272, y=401, height=90)
base.mainloop()